In [1]:
import yfinance as yf
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')


from data_downloader import DataDownloader
from var_model import VAR
from dice_model import DiceModel

In [2]:
downloader = DataDownloader()

assets = ['AAPL', 'IBM', 'TSLA', 'GOOG', 'NVDA'] 
benchmark = '^GSPC'  
start_date = '2019-01-01'
end_date = '2023-12-31'
rf = .065

asset_prices, benchmark_prices = downloader.download_data(start_date=start_date, end_date=end_date,
                                                                      assets=assets, benchmark=benchmark)

[*********************100%%**********************]  5 of 5 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
asset_prices.head()

Ticker,AAPL,GOOG,IBM,NVDA,TSLA
Date,,,,,
2019-01-02,37.750088,52.164391,83.947762,3.378323,20.674667
2019-01-03,33.989899,50.678543,82.271866,3.174214,20.024000
2019-01-04,35.440899,53.404346,85.485214,3.377579,21.179333
2019-01-07,35.362019,53.288631,86.089989,3.556391,22.330667
2019-01-08,36.036125,53.682163,87.314117,3.467853,22.356667


In [26]:
# Cargar archivos CSV
temperature = pd.read_csv('Data/Temperature.csv')
drought = pd.read_csv('Data/Drought_Severity.csv')

# Cargar archivo XLSX
co2_emission = pd.read_excel('Data/CO2_Emissions.xlsx', skiprows=10).drop(index=0)
co2_emission = co2_emission[["Month",
                             "Coal, Including Coal Coke Net Imports, CO2 Emissions",
                             "Natural Gas, Excluding Supplemental Gaseous Fuels, CO2 Emissions", 
                             "Petroleum, Excluding Biofuels, CO2 Emissions", 
                             "Total Energy CO2 Emissions"]]
co2_emission = co2_emission.rename(columns={
    "Month": "Date",
    "Coal, Including Coal Coke Net Imports, CO2 Emissions": "Coal",
    "Natural Gas, Excluding Supplemental Gaseous Fuels, CO2 Emissions": "Natural Gas",
    "Petroleum, Excluding Biofuels, CO2 Emissions": "Petroleum",
    "Total Energy CO2 Emissions": "Total CO2 Emissions"
})

co2_emission

,Date,Coal,Natural Gas,Petroleum,Total CO2 Emissions
1,1973-01-01,109.552,125.557,218.972,454.082
2,1973-02-01,98.833,113.75,202.519,415.102
3,1973-03-01,98.483,107.7,201.245,407.428
4,1973-04-01,94.15,97.838,176.222,368.21
5,1973-05-01,95.424,92.039,188.57,376.032
...,...,...,...,...,...
613,2024-01-01,83.031,200.228,183.995,467.886
614,2024-02-01,53.298,160.596,173.319,387.805
615,2024-03-01,46.63,150.845,185.759,383.866
616,2024-04-01,44.366,129.155,184.741,358.875


In [27]:
# Function to drop a or multiple columns from a dataframe
def drop_columns(df, columns):
    df = df.drop(columns=columns)   
    return df

# Function to rename columns in a dataframe
def rename_columns(df, columns):
    df = df.rename(columns=columns)
    return df

# Drop the 'Average surface temperature.1', 'Entity', 'year'  columns from the temperature dataframe and also index
temperature = drop_columns(temperature, ['Average surface temperature.1','Code', 'Entity', 'year'])

# Rename the columns in the temperature dataframe 
temperature = rename_columns(temperature, {'Day': 'Date',
                             'Average surface temperature': 'Temperature'})

temperature

,Date,Temperature
0,15/01/40,-7.278521
1,15/02/40,-3.639929
2,15/03/40,0.422743
3,15/04/40,6.756010
4,15/05/40,13.282525
...,...,...
1011,15/04/24,9.773721
1012,15/05/24,14.883865
1013,15/06/24,20.794046
1014,15/07/24,22.566105


In [28]:
# Drop any columns that are not needed (example: 'Code' column if it exists)
if 'Code' in drought.columns:
    drought = drought.drop(columns=['Code'])

# Remove the 'Name' column
if 'Name' in drought.columns:
    drought = drought.drop(columns=['Name'])

# Change the name of the 'MapDate' column to 'Date' in the drought dataset
drought = rename_columns(drought, {'MapDate': 'Date'})

# Convert the 'Date' column to datetime format
drought['Date'] = pd.to_datetime(drought['Date'], format='%Y%m%d')

drought

,Date,DSCI
0,2014-12-30,104
1,2015-01-06,100
2,2015-01-13,101
3,2015-01-20,103
4,2015-01-27,109
...,...,...
503,2024-08-20,89
504,2024-08-27,104
505,2024-09-03,107
506,2024-09-10,117
